# Overview:
In this notebook we look at how we can run code on worker nodes. This is useful fro various troubleshooting, debuggine, and general engineering tasks.

# 1. Create SparkContext

In [1]:
spark_app_name = "spark-jupyter-worker-dev"
docker_image = "tschneider/apache-spark-k8:v7"
k8_master_ip = "15.4.7.11"

import spark_helper
spark_session = spark_helper.create_spark_session(spark_app_name, docker_image, k8_master_ip)
sc = spark_session.sparkContext

Setting SPARK_HOME
/usr/lib/spark-3.1.1-bin-hadoop2.7

Running findspark.init() function
['/usr/lib/spark-3.1.1-bin-hadoop2.7/python', '/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip', '/ml-training-jupyter-notebooks/Machine Learning/Big Data And Big Compute/Apache Spark', '/usr/local/lib/python39.zip', '/usr/local/lib/python3.9', '/usr/local/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/site-packages', '/ml-training-jupyter-notebooks/Utilities']

Setting PYSPARK_PYTHON
/usr/local/bin/python3

Configuring URL for kubernetes master
k8s://https://15.4.7.11:6443

Determining IP Of Server
The ip was detected as: 15.4.12.12

Creating SparkConf Object
('spark.master', 'k8s://https://15.4.7.11:6443')
('spark.app.name', 'spark-jupyter-worker-dev')
('spark.submit.deploy.mode', 'cluster')
('spark.kubernetes.container.image', 'tschneider/apache-spark-k8:v7')
('spark.kubernetes.namespace', 'spark')
('spark.kubernetes.pyspark.pythonVersion', '3')
('spark.kubernetes.au

22/01/21 18:34:05 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 15.4.12.12 instead (on interface eth0)
22/01/21 18:34:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/21 18:34:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).



Done!


In [2]:
! kubectl -n spark get pods

NAME                                               READY     STATUS    RESTARTS   AGE
spark-jupyter-worker-dev-4f117e7e7deada7f-exec-1   1/1       Running   0          25s
spark-jupyter-worker-dev-4f117e7e7deada7f-exec-2   1/1       Running   0          24s
spark-jupyter-worker-dev-4f117e7e7deada7f-exec-3   1/1       Running   0          24s


# 2. Run Functions On Workers

## 2.1. The Parallelize() function
The easiest way to do this is use the parallelize function on the SparkContext object. This utility function will run an arbitrary function N times, in parallel, on the cluster. It makes no guarantee about when and where functions run but it is helpful for a number of quick tasks.

In [4]:
worker_count = int(spark_session.sparkContext.getConf().get('spark.executor.instances'))
rdd = spark_session.sparkContext.parallelize(range(worker_count)).map(lambda var: var)
results = rdd.collect()
results

[0, 1, 2]

We can see that the function being parallelized needs to accept an index number and can return results which will be aggregated into a spark dataframe.

Setting the range equal to the number of workers is a good way to ensure the arbitrary function runs once on each worker (assuming they are equally busy at the time).

Getting a bit more advanced we can see how this utility can be used to interrogate or modify the workers.

In [8]:
def get_hostname(var):
    import socket
    hostname = socket.gethostname()
    return hostname
    
rdd = spark_session.sparkContext.parallelize(range(worker_count)).map(get_hostname)
results = rdd.collect()
results

['spark-jupyter-worker-dev-4f117e7e7deada7f-exec-1',
 'spark-jupyter-worker-dev-4f117e7e7deada7f-exec-3',
 'spark-jupyter-worker-dev-4f117e7e7deada7f-exec-2']

We can see we have gotten the hostnames of all the workers.

We will also notice that print statements do not show up. This is because the STDOUT of the worker is not being captured and sent back to the driver. If we want this to work, we need a more advanced solution like logging to a network bus or a database.

In [11]:
rdd = spark_session.sparkContext.parallelize(range(worker_count)).map(lambda var: print(var))
results = rdd.collect()
results

[None, None, None]

# 3. Things You Cannot Do On Workers

## 3.1. We Cannot Create A SparkContext or SparkSession 

In [16]:
def do_spark_work(var):
    import pyspark
    import netifaces
    import re
    
    print("Determine the ip address of the machine")
    nic_uuid = netifaces.gateways()['default'][netifaces.AF_INET][1]
    nic_details = netifaces.ifaddresses(nic_uuid)
    ip_address = None
    for i, nic_detail, in nic_details.items():
        if all([key in nic_detail[0].keys() for key in ["addr", "netmask", "broadcast"]]):
            if re.match("([0-9]+\\.)+", nic_detail[0]["addr"]):
                ip_address = nic_detail[0]["addr"]
                break
    if not ip_address:
        raise Exception("Unable to determine ip address.")

    print("Setting parameters for the cluster")
    spark_app_name = "spark-jupyter-mlib"
    docker_image = "tschneider/pyspark:v5"
    k8_master_ip = "15.4.7.11"
    kubernetes_master_ip = k8_master_ip
    kubernetes_master_port = "6443"
    spark_master_url = "k8s://https://{0}:{1}".format(kubernetes_master_ip, kubernetes_master_port)

    print("Creating SparkConf Object")
    sparkConf = pyspark.SparkConf()
    sparkConf.setMaster(spark_master_url)
    sparkConf.setAppName(spark_app_name)
    sparkConf.set("spark.submit.deploy.mode", "cluster")
    sparkConf.set("spark.kubernetes.container.image", docker_image) 
    sparkConf.set("spark.kubernetes.namespace", "spark")
    sparkConf.set("spark.kubernetes.pyspark.pythonVersion", "3")
    sparkConf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark-sa")
    sparkConf.set("spark.kubernetes.authenticate.serviceAccountName", "spark-sa")
    sparkConf.set("spark.executor.instances", "3")
    sparkConf.set("spark.executor.cores", "2")
    sparkConf.set("spark.executor.memory", "512m")
    sparkConf.set("spark.executor.memoryOverhead", "512m")
    sparkConf.set("spark.driver.memory", "512m")
    sparkConf.set("spark.driver.host", ip_address)
    sparkConf.set("spark.files.overwrite", "true")
    sparkConf.set("spark.files.useFetchCache", "false")

    print("Creating SparkSession Object")
    from pyspark.sql import SparkSession
    spark_session = SparkSession.builder.config(conf=sparkConf).getOrCreate()
    
    return spark_session
    
rdd = spark_session.sparkContext.parallelize(range(worker_count)).map(do_spark_work)
results = rdd.collect()
results

22/01/21 18:51:09 WARN TaskSetManager: Lost task 1.0 in stage 12.0 (TID 99) (10.36.0.2 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_6987/3495222781.py", line 47, in do_spark_work
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/session.py", line 228, in getOrCreate
    sc = SparkContext.getOrCreate(sparkConf)
  File "/usr/lib/sp

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 12.0 failed 4 times, most recent failure: Lost task 1.3 in stage 12.0 (TID 110) (10.36.0.2 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_6987/3495222781.py", line 47, in d

## 3.2. We Cannot Touch A SparkContext Or Spark Session

In [21]:
def foobar(spark_session):
    return spark_session.SparkContext

rdd = spark_session.sparkContext.parallelize(range(worker_count)).map(
        lambda var: foobar(spark_session))
results = rdd.collect()
results

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.9/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.9/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
  File "/usr/local/lib/python3.9/site-packages/pyspark/context.py", line 353, in __getnewargs__
    raise Exception(
Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

## 3.3. We Cannot Interact With Unserializable Objects

In [22]:
def foobar(o):
    return o

import threading
lock = threading.Lock()

rdd = spark_session.sparkContext.parallelize(range(worker_count)).map(
        lambda var: foobar(lock))
results = rdd.collect()
results

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.9/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.9/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.lock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.lock' object